In [3]:
#Loading libraries
import pandas as pd
import numpy as np
import random
import psycopg2

In [4]:
#Loading DB with municipalities

HOST="db-bdt.crc01ueoqr2u.eu-west-1.rds.amazonaws.com"
DB="postgres"
USER="bdtuser"
PASSWORD="BDTdatabase2022"

try:
    conn = psycopg2.connect(
        host=HOST,
        database=DB,
        user=USER,
        password=PASSWORD)
            
    cur = conn.cursor()
    
    sql = "SELECT * FROM municipalities AS m"
    cur.execute(sql)
    prov = cur.fetchall()
    # close communication with the database
    cur.close()
except (Exception, psycopg2.DatabaseError) as error:
    print(error)
finally:
    if conn is not None:
        conn.close()

In [5]:
#Saving loaded DB as pandas DF
comunes = pd.DataFrame(prov)
#Naming the columns
comunes.columns = ['comune','province','Region','population','lat_lng','istat_code','6']

In [6]:
comunes.head()

,comune,province,Region,population,lat_lng,istat_code,6
0,Andezeno,TO,Piemonte,1987.0,"(45.03586095,7.86994864)",1009,0.51
1,Andrate,TO,Piemonte,520.0,"(45.52837479,7.87383937)",1010,0.76
2,Angrogna,TO,Piemonte,872.0,"(44.84344991,7.22429215)",1011,0.45
3,Arignano,TO,Piemonte,1080.0,"(45.04402899,7.90244635)",1012,0.77
4,Avigliana,TO,Piemonte,12262.0,"(45.07935742,7.39632339)",1013,0.76


In [7]:
#Loading DB with Activity Data (ISTAT-based)
HOST="db-bdt.crc01ueoqr2u.eu-west-1.rds.amazonaws.com"
DB="postgres"
USER="bdtuser"
PASSWORD="BDTdatabase2022"

try:
    conn = psycopg2.connect(
        host=HOST,
        database=DB,
        user=USER,
        password=PASSWORD)
            
    cur = conn.cursor()
    
    sql = "SELECT * FROM sport_activity as s"
    cur.execute(sql)
    prov = cur.fetchall()
    # close communication with the database
    cur.close()
except (Exception, psycopg2.DatabaseError) as error:
    print(error)
finally:
    if conn is not None:
        conn.close()

In [8]:
#Saving loaded DB as pandas DF
activitydf = pd.DataFrame(prov)
activitydf.columns = ['Region','score','dev_st','Daily_steps','steps_sd','Daily_pushups','pushups_sd','Daily_squats','squats_sd',"Daily_gym","Daily_gym_sd","Daily_swimming","Daily_swimming_sd"]

In [9]:
activitydf.head()

,Region,score,dev_st,Daily_steps,steps_sd,Daily_pushups,pushups_sd,Daily_squats,squats_sd,Daily_gym,Daily_gym_sd,Daily_swimming,Daily_swimming_sd
0,Trentino Alto Adige,0.599,0.003,6345.0,32.0,25.0,1.0,47.0,1.0,0.51,0.15,0.51,0.15
1,Valle d'Aosta,0.53,0.002,5614.0,21.0,22.0,1.0,41.0,1.0,0.45,0.14,0.45,0.14
2,Veneto,0.5005,0.012,5301.0,127.0,21.0,2.0,39.0,2.0,0.43,0.15,0.43,0.15
3,Lombardia,0.4965,0.009,5259.0,95.0,21.0,1.0,39.0,2.0,0.43,0.15,0.43,0.15
4,Emilia-Romagna,0.477,0.01,5052.0,106.0,20.0,1.0,37.0,2.0,0.41,0.15,0.41,0.15


In [10]:
#Saving data about specific activities into separated datasets

#Squat data 
squats = pd.DataFrame()
#squats['Region']=activitydf.Region
squats['Daily_squats']=activitydf.Daily_squats
squats['squats_sd']=activitydf.squats_sd

#Pushup data
pushups = pd.DataFrame()
pushups['Daily_pushups'] = activitydf['Daily_pushups']
pushups['pushups_sd'] = activitydf['pushups_sd']
#Steps data
steps = pd.DataFrame()
steps['Daily_steps'] = activitydf['Daily_steps']
steps['steps_sd'] = activitydf['steps_sd']

In [11]:
#Associating each region with its position code

codreg = dict(enumerate(activitydf.Region))

In [12]:
#Changing name of Trentino to adapt it to the municipalities DB

comunes.loc[comunes.Region=='Trentino-Alto Adige','Region'] = 'Trentino Alto Adige'

# Gaussian simulation of data
**STEPS, SQUATS AND PUSHUPS**

In [13]:
#Gaussian simulation of daily steps data
retsteps = []
strec = steps.to_records(index=False)
for i in range(len(strec)):
    #regional mean
    mu = strec[i][0]
    #regional sd
    sigma = strec[i][1]
    #regional activity score to weight population
    regscore = float(activitydf[activitydf.Region==codreg[i]]['score'])
    ix = list()
    #Doing it for each comune
    popolaz = comunes[comunes.Region==codreg[i]].population
    for popul in popolaz:
        n_people = int(popul)*regscore
        n_people = int(np.sqrt(n_people))
        s = np.random.normal(mu,sigma,n_people)
        s = s.astype(int)
        ix.extend(s)
    retsteps.extend(ix) #prima era .append

In [14]:
#Comune per case
retcomuni = []
for i in range(len(strec)):
    #regional activity score to weight population
    regscore = float(activitydf[activitydf.Region==codreg[i]]['score'])

    #Doing it for each comune
    propcom = []
    popolaz = comunes[comunes.Region==codreg[i]].population
    #Weights of the comunes of the regions
    wreg = []
    for popul in popolaz:
        n_people = int(popul)*regscore
        n_people = int(np.sqrt(n_people))
        propcom.append(n_people)
    icomuni = list(comunes[comunes.Region==codreg[i]].comune)
    for en in range(len(icomuni)):
        comweight = [icomuni[en]]*propcom[en]
        wreg.extend(comweight)
        
    #retcomuni.append(wreg)
    retcomuni.extend(wreg)
        
        

In [17]:
len(retsteps)

329597

In [18]:
#Gaussian simulation of daily squats data
prop = 0.156/2
retsquats = []
sqrec = squats.to_records(index=False)
for i in range(len(sqrec)):
    #regional mean
    mu = sqrec[i][0]
    #regional sd
    sigma = sqrec[i][1]
    #regional activity score to weight population
    regscore = float(activitydf[activitydf.Region==codreg[i]]['score'])
    ix = list()
    #Doing it for each comune
    popolaz = comunes[comunes.Region==codreg[i]].population
    for popul in popolaz:
        n_people = int(popul)*regscore*prop
        n_people = int(np.sqrt(n_people))
        s = np.random.normal(mu,sigma,n_people)
        s = s.astype(int)
        ix.extend(s)
    retsquats.extend(ix) #prima era .append

In [19]:
#Comune per case - squats
retcomunis = []
for i in range(len(sqrec)):
    #regional activity score to weight population
    regscore = float(activitydf[activitydf.Region==codreg[i]]['score'])

    #Doing it for each comune
    propcom = []
    popolaz = comunes[comunes.Region==codreg[i]].population
    #Weights of the comunes of the regions
    wreg = []
    for popul in popolaz:
        n_people = int(popul)*regscore*prop
        n_people = int(np.sqrt(n_people))
        propcom.append(n_people)
    icomuni = list(comunes[comunes.Region==codreg[i]].comune)
    for en in range(len(icomuni)):
        comweight = [icomuni[en]]*propcom[en]
        wreg.extend(comweight)
        
    #retcomuni.append(wreg)
    retcomunis.extend(wreg)
        

In [20]:
#Gaussian simulation of daily pushups data
prop = 0.156/2
retpushups = []
purec = pushups.to_records(index=False)
for i in range(len(sqrec)):
    #regional mean
    mu = purec[i][0]
    #regional sd
    sigma = purec[i][1]
    #regional activity score to weight population
    regscore = float(activitydf[activitydf.Region==codreg[i]]['score'])
    ix = list()
    #Doing it for each comune
    popolaz = comunes[comunes.Region==codreg[i]].population
    for popul in popolaz:
        n_people = int(popul)*regscore*prop
        n_people = int(np.sqrt(n_people))
        s = np.random.normal(mu,sigma,n_people)
        s = s.astype(int)
        ix.extend(s)
    retpushups.extend(ix) #prima era .append

In [21]:
#Generating conventional device IDs
idsteps = ["D_"+str(i) for i in range(len(retsteps))]

In [22]:
#Generating conventional device IDs
idsquats = ["D_"+str(i) for i in range(len(retsquats))]
idspushups = ["D_"+str(i) for i in range(len(retpushups))]

In [23]:
stepsdailylogs = list(zip(idsteps,retsteps,retcomuni))
np.random.shuffle(stepsdailylogs)

In [24]:
squatsdailylogs = list(zip(idsquats,retsquats,retcomunis))
np.random.shuffle(squatsdailylogs)

In [25]:
pushupsdailylogs = list(zip(idspushups,retpushups,retcomunis))
np.random.shuffle(pushupsdailylogs)

In [26]:
pushupsdailylogs[29:41]

[('D_61780', 17, 'Città di Castello'),
 ('D_16908', 19, 'Inveruno'),
 ('D_54399', 19, 'Piandimeleto'),
 ('D_43623', 15, 'Cosseria'),
 ('D_35884', 19, 'Fosdinovo'),
 ('D_76876', 13, 'Bovalino'),
 ('D_66291', 16, 'Baunei'),
 ('D_4752', 23, 'Veronella'),
 ('D_56136', 17, 'Monteprandone'),
 ('D_27378', 19, 'Barzanò'),
 ('D_57754', 19, 'Turania'),
 ('D_30764', 18, 'Fontevivo')]

# Definition of JSON with simulated steps

In [27]:
#Generating the list that will become a JSON FILE
jsonlisteps = []
for (x,y,z) in stepsdailylogs:
    dic = {}
    dic['id'] = x
    dic['steps'] = y
    dic['lat'] = ""
    dic['lng'] = ""
    dic['comune'] = z
    jsonlisteps.append(dic)

In [30]:
len(jsonlisteps)

329597

   # Definition of JSON with simulated squats

In [92]:
#Generating the list that will become a JSON FILE
jsonlisquats = []
for (x,y,z) in squatsdailylogs:
    dic = {}
    dic['id'] = x
    dic['squats'] = y
    dic['lat'] = ""
    dic['lng'] = ""
    dic['comune'] = z
    jsonlisquats.append(dic)

In [93]:
jsonlisquats[:5]

[{'id': 'D_1725983',
  'squats': 22,
  'lat': '',
  'lng': '',
  'comune': 'Altofonte'},
 {'id': 'D_1832128', 'squats': 21, 'lat': '', 'lng': '', 'comune': 'Caserta'},
 {'id': 'D_803015', 'squats': 36, 'lat': '', 'lng': '', 'comune': 'Stazzema'},
 {'id': 'D_1365029', 'squats': 36, 'lat': '', 'lng': '', 'comune': 'Tivoli'},
 {'id': 'D_529310', 'squats': 39, 'lat': '', 'lng': '', 'comune': 'Siziano'}]

In [86]:
len(jsonlisquats)

1951017

  # Definition of JSON with simulated pushups

In [90]:
#Generating the list that will become a JSON FILE
jsonlispushups = []
for (x,y,z) in pushupsdailylogs:
    dic = {}
    dic['id'] = x
    dic['pushups'] = y
    dic['lat'] = ""
    dic['lng'] = ""
    dic['comune'] = z
    jsonlispushups.append(dic)

In [91]:
jsonlispushups[:5]

[{'id': 'D_1305331', 'pushups': 19, 'lat': '', 'lng': '', 'comune': 'Roma'},
 {'id': 'D_426504', 'pushups': 19, 'lat': '', 'lng': '', 'comune': 'Albino'},
 {'id': 'D_1419114',
  'pushups': 17,
  'lat': '',
  'lng': '',
  'comune': 'Corciano'},
 {'id': 'D_1141039',
  'pushups': 19,
  'lat': '',
  'lng': '',
  'comune': 'San Marzano Oliveto'},
 {'id': 'D_770619',
  'pushups': 19,
  'lat': '',
  'lng': '',
  'comune': 'Santa Sofia'}]

# Saving CSVs to do a trial in R

In [99]:
datasteps = pd.DataFrame(jsonlisteps)
stepsaggregated = datasteps.groupby('comune').mean()

In [98]:
datasteps

,id,steps,lat,lng,comune
0,D_15120317,4622,,,Pesaro
1,D_5031805,5258,,,Milano
2,D_1382762,5037,,,Pove del Grappa
3,D_16891482,4440,,,Roma
4,D_23766798,2969,,,Puglianello
...,...,...,...,...,...
25059468,D_4197358,5307,,,Cologno Monzese
25059469,D_22765367,3077,,,Licata
25059470,D_22670206,3010,,,Sant'Agata di Militello
25059471,D_1043990,5184,,,Verona


In [ ]:
datasquats = pd.DataFrame(jsonlisquats)
squatsaggregated = datasquats.groupby('comune').mean()

In [ ]:
datapushups = pd.DataFrame(jsonlispushups)
pushupsaggregated = datapushups.groupby('comune').mean()

In [ ]:
stepsaggregated.to_csv("stepsaggregated.csv")
squatsaggregated.to_csv("squatsaggregated.csv")
pushupsaggregated.to_csv("pushupsaggregated.csv")